In [1]:
# Montando Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Definindo diretório padrão deste notebook
import sys
base_path = "/content/drive/MyDrive/VAE_TCC"
sys.path.append(base_path)

In [ ]:
# Instalando bibliotecas auxiliares
#!pip install tensorflow
#!pip install soundfile
#!pip install auraloss
#!pip install librosa

In [ ]:
# Importando bibliotecas auxiliares
import time
import numpy as np
import soundfile as sf
import tensorflow as tf
from tensorflow.keras.backend import clear_session

In [ ]:
# Importando arquivos auxiliares
import utils
import configs
from model import VAE_GAN
from audio_info import AudioInfo

In [ ]:
# Re-carregando arquivos para garantia de alterações
import importlib
importlib.reload(utils)
importlib.reload(configs)

<module 'configs' from '/content/drive/MyDrive/VAE_TCC/configs.py'>

In [ ]:
start_time = time.time()
current_id = utils.generate_random_id()
print(f"[ID do Experimento] - {current_id}")

[ID do Experimento] - ZF8046


In [ ]:
# Carregando os dados de áudio e instanciando otimizador
audio_info = AudioInfo(audio_path=configs.AUDIO_PATH, duration=configs.AUDIO_DURATION, sr=configs.AUDIO_RATE)

In [ ]:
# Extraindo e preparando dados de Treinamento
data = audio_info.get_audio_data(num_audio_samples=configs.NUM_AUDIO_SAMPLES)
multiband_dataset = []
for i in range(data.shape[0]):
    subbands = audio_info.multiband_decomposition(data[i], num_bands=configs.NUM_BANDS)
    multiband_dataset.append(subbands)

data = np.array(multiband_dataset)
print(f"[Dados de Treinamento] - Formato: {data.shape}")

[Extraindo formas de onda]
 - pop.00058: OK
 - pop.00055: OK
 - pop.00049: OK
 - pop.00059: OK
 - pop.00061: OK
 - pop.00033: OK
 - pop.00032: OK
 - pop.00019: OK
 - pop.00011: OK
 - pop.00022: OK
 - pop.00004: OK
 - pop.00082: OK
 - pop.00026: OK
 - pop.00053: OK
 - pop.00009: OK
 - pop.00016: OK
 - pop.00025: OK
 - pop.00000: OK
 - pop.00052: OK
 - pop.00035: OK
 - pop.00068: OK
 - pop.00038: OK
 - pop.00095: OK
 - pop.00067: OK
 - pop.00012: OK
 - pop.00065: OK
 - pop.00084: OK
 - pop.00087: OK
 - pop.00037: OK
 - pop.00018: OK
 - pop.00093: OK
 - pop.00078: OK
 - pop.00064: OK
 - pop.00096: OK
 - pop.00051: OK
 - pop.00034: OK
 - pop.00003: OK
 - pop.00080: OK
 - pop.00005: OK
 - pop.00054: OK
 - pop.00085: OK
 - pop.00073: OK
 - pop.00081: OK
 - pop.00028: OK
 - pop.00047: OK
 - pop.00062: OK
 - pop.00060: OK
 - pop.00066: OK
 - pop.00077: OK
 - pop.00090: OK
 - pop.00048: OK
 - pop.00092: OK
 - pop.00006: OK
 - pop.00002: OK
 - pop.00071: OK
 - pop.00014: OK
 - pop.00070: OK
 - p

In [ ]:
# Inicializando o modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=configs.LEARNING_RATE, beta_1=configs.BETA_1,
                                     beta_2=configs.BETA_2, epsilon=configs.EPSILON)
model = VAE_GAN(input_shape=data.shape, latent_dim=configs.LATENT_DIM, hidden_dims=configs.VAE_HIDDEN_DIMS, id=current_id,
            duration=configs.AUDIO_DURATION, rate=configs.AUDIO_RATE, kernel_sizes=configs.VAE_KERNELS, strides=configs.VAE_STRIDES,
             loud_stride=configs.LOUD_STRIDE, kl_beta=configs.KL_BETA, batch_size=configs.BATCH_SIZE, num_bands=configs.NUM_BANDS,
            residual_depth=configs.RESIDUAL_DEPTH, use_noise=configs.USE_NOISE)


[Incializando VAE-GAN]
	[CONSTRUINDO ENCODER]


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


		 - Forma de saída MU:  (None, 1, 128)
		 - Forma de saída LOG_VAR:  (None, 1, 128)

	[CONSTRUINDO DECODER]
	[INFO] Padding necessário para equivalência de saídas. (27392 != 27500)
		 - Forma de saída DECODER:  (None, 16, 27500)
[VAE-GAN Inicializado]


In [ ]:
# Visualizando modelo
model.summary()

Model: "vae_gan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Encoder (Functional)                 │ [(None, 1, 128), (None, 1,  │      15,438,192 │
│                                      │ 128)]                       │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Decoder (Functional)                 │ (None, 16, 27500)           │      22,444,610 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 37,882,802 (144.51 MB)

 Trainable params: 37,826,906 (144.30 MB)

 Non-trainable params: 55,896 (218.34 KB)

In [ ]:
model.encoder.summary()

Model: "Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 16, 27500)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 16, 27500)      │        110,000 │ input_layer[0][0]      │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu (LeakyReLU)   │ (None, 16, 27500)      │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 4, 64)          │     14,080,064 │ leaky_re_lu[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 4, 64)          │            256 │ conv1d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_1 (LeakyReLU) │ (None, 4, 64)          │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 1, 128)         │         65,664 │ leaky_re_lu_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 1, 128)         │            512 │ conv1d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_2 (LeakyReLU) │ (None, 1, 128)         │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 1, 256)         │        262,400 │ leaky_re_lu_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 1, 256)         │          1,024 │ conv1d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_3 (LeakyReLU) │ (None, 1, 256)         │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_3 (Conv1D)         │ (None, 1, 512)         │        524,800 │ leaky_re_lu_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_4 (LeakyReLU) │ (None, 1, 512)         │              0 │ conv1d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_4 (Conv1D)         │ (None, 1, 128)         │        327,808 │ leaky_re_lu_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_5 (Conv1D)         │ (None, 1, 128)         │         65,664 │ leaky_re_lu_4[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 15,438,192 (58.89 MB)

 Trainable params: 15,382,296 (58.68 MB)

 Non-trainable params: 55,896 (218.34 KB)

In [ ]:
model.decoder.summary()

Model: "Decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 1, 128)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 1, 109568)      │     14,134,272 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 214, 512)       │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ upsampling_block          │ (None, 856, 512)       │              0 │ reshape[0][0]          │
│ (UpsamplingBlock)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ residual_block            │ (None, 856, 512)       │      6,292,992 │ upsampling_block[0][0] │
│ (ResidualBlock)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ upsampling_block_1        │ (None, 3424, 256)      │              0 │ residual_block[0][0]   │
│ (UpsamplingBlock)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ residual_block_1          │ (None, 3424, 256)      │      1,573,632 │ upsampling_block_1[0]… │
│ (ResidualBlock)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ upsampling_block_2        │ (None, 13696, 128)     │              0 │ residual_block_1[0][0] │
│ (UpsamplingBlock)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ residual_block_2          │ (None, 13696, 128)     │        393,600 │ upsampling_block_2[0]… │
│ (ResidualBlock)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ upsampling_block_3        │ (None, 27392, 64)      │              0 │ residual_block_2[0][0] │
│ (UpsamplingBlock)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ residual_block_3          │ (None, 27392, 64)      │         49,344 │ upsampling_block_3[0]… │
│ (ResidualBlock)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_18 (Conv1D)        │ (None, 27392, 1)       │            449 │ residual_block_3[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_19 (Conv1D)        │ (None, 27392, 1)       │            193 │ residual_block_3[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply (Multiply)       │ (None, 27392, 1)       │              0 │ conv1d_18[0][0],       │
│                           │                        │                │ conv1d_19[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_20 (Conv1D)   

 Total params: 22,444,610 (85.62 MB)

 Trainable params: 22,444,610 (85.62 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Treinando a representação do modelo
clear_session()
signal_losses, kl_losses = model.train(data, configs.EPOCHS, optimizer)

[Iniciando Treinamento de Representação]
[INFO] Dividindo dataset em batches para o treinamento.
[INFO] Dataset dividido em batches de tamanho 8 para o treinamento.

# [ Época 1 | Loss: 673.0504684 |  Recon. Loss: 672.8544998 | KL Loss: 1.9596469]
# [ Época 16 | Loss: 586.882225 |  Recon. Loss: 586.7473946 | KL Loss: 1.3483536]
# [ Época 31 | Loss: 579.3168526 |  Recon. Loss: 579.1071472 | KL Loss: 2.0970709]
# [ Época 46 | Loss: 575.0065308 |  Recon. Loss: 574.8296967 | KL Loss: 1.7683347]
# [ Época 61 | Loss: 570.9011192 |  Recon. Loss: 570.6909485 | KL Loss: 2.1017462]
# [ Época 76 | Loss: 566.4673424 |  Recon. Loss: 566.2346954 | KL Loss: 2.3265136]
# [ Época 91 | Loss: 563.8622932 |  Recon. Loss: 563.6293907 | KL Loss: 2.3290488]
# [ Época 106 | Loss: 562.3856201 |  Recon. Loss: 562.1644096 | KL Loss: 2.212087]
# [ Época 121 | Loss: 560.8890915 |  Recon. Loss: 560.6207886 | KL Loss: 2.6830403]
# [ Época 136 | Loss: 559.4028702 |  Recon. Loss: 559.1079712 | KL Loss: 2.9488833]
# [ 

In [ ]:
# Treinando o modelo com adversarial fine-tuning
clear_session()
gen_optimizer = tf.keras.optimizers.Adam(learning_rate=configs.LEARNING_RATE, beta_1=configs.BETA_1,
                                     beta_2=configs.BETA_2, epsilon=configs.EPSILON)
discr_optimizer = tf.keras.optimizers.Adam(learning_rate=configs.LEARNING_RATE, beta_1=configs.BETA_1,
                                     beta_2=configs.BETA_2, epsilon=configs.EPSILON)
generator_losses, discriminator_losses = model.train_gan(data, configs.DISCR_EPOCHS, gen_optimizer, discr_optimizer,
                                                         configs.DISCR_HIDDEN_DIMS, configs.DISCR_KERNELS, configs.DISCR_STRIDES)

In [ ]:
end_time = time.time()
execution_time = end_time - start_time
execution_time = str(round(execution_time, 2))

In [ ]:
# Avaliando espaço latente
# reduced_latent = model.compact_latent_representation(data)

In [ ]:
# Obtendo medias MU para visualização do espaço latente
mu = model.encode(data)[0]
mu = mu.numpy()

In [ ]:
utils.show_results(execution_time)
#utils.save_metadata(current_id, execution_time)
utils.save_graphs(current_id, signal_losses, kl_losses, generator_losses, discriminator_losses, mu, configs.COMPACT_LATENT_SPACE)

In [ ]:
# Gerando N formas de ondas a partir do espaço latente
generated = model.sample(configs.NUM_SAMPLES_GENERATE, data, configs.COMPACT_LATENT_SPACE)

In [ ]:
# Para cada espectrograma gerado, salva o resultado em formato de áudio e em um arquivo txt
for i in range(configs.NUM_SAMPLES_GENERATE):
    curr_gen = generated[i]
    curr_gen = audio_info.multiband_synthesis(curr_gen, configs.NUM_BANDS)
    file_result = base_path + '/results/generated_audio_' + current_id + '_' + str(i+1) + '.ogg'
    sf.write(file_result, curr_gen, configs.AUDIO_RATE)
    print(f"[ Salvando resultado gerado {i + 1} ] - {current_id}")